In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV

In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 0C74-9C02

 Directory of C:\Users\Jory Anderson\Desktop\Programming\Repos\Loan-Default\notebooks

2020-07-08  04:11 PM    <DIR>          .
2020-07-08  04:11 PM    <DIR>          ..
2020-07-08  03:21 AM    <DIR>          .ipynb_checkpoints
2019-12-11  03:58 AM        26,567,651 application_test.csv
2019-12-11  03:58 AM       166,133,370 application_train.csv
2020-07-08  04:07 PM            58,277 Data_Exploration.ipynb
2020-07-08  04:11 PM            52,596 Data_Exploration_FullSet.ipynb
2020-07-07  08:51 PM       477,730,169 full.csv
2020-07-08  04:09 PM           231,021 Model Performance.ipynb
2020-07-07  08:47 PM        12,657,678 select.csv
               7 File(s)    683,430,762 bytes
               3 Dir(s)  34,024,292,352 bytes free


In [3]:
dataset = pd.read_csv("application_train.csv")

In [4]:
dataset.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
dataset.shape

(307511, 122)

In [6]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Get  columns whose data type is object i.e. string
filteredColumns = dataset.dtypes[dataset.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

# Label Encoding if there are two categorical labels on a feature)
for col in listOfColumnNames:
    if dataset.dtypes[col] == np.object:
        # If 2 or fewer unique categories
        if len(list(dataset[col].unique())) <= 2:
            # Train on the training data
            le.fit(dataset[col])
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

# One-Hot Encoding (> 2 categorical labels for a feature)
num_dataset = pd.get_dummies(dataset)

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
3 columns were label encoded.


In [7]:
null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,1,0,0,0,0,0,0,1,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,0,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,...,0,0,0,0,0,0,1,0,1,0
307507,456252,0,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,...,0,0,0,0,0,0,1,0,1,0
307508,456253,0,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,...,0,0,0,0,0,1,0,0,1,0
307509,456254,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,...,0,0,0,0,0,0,1,0,1,0


In [10]:
null_rows.shape

(296160, 246)

What percent of the rows in the data have a null value in at least one column?

In [11]:
null_rows.shape[0]/dataset.shape[0]

0.9630874993089678

97% of the rows have at least one column with a null value. 
How will we handle null values?

In [12]:
dataset.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [13]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(num_dataset)
new_num_dataset = pd.DataFrame(imp.transform(num_dataset))
new_num_dataset.columns = num_dataset.columns
new_num_dataset.index = num_dataset.index
num_dataset = new_num_dataset

null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes


In [14]:
num_dataset.shape

(307511, 246)

In [15]:
num_dataset.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002.0,1.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,100003.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,100004.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(num_dataset.drop('TARGET', 1), 
                                    num_dataset['TARGET'], test_size = .2, random_state=42,shuffle=True)

In [17]:
# SMOTE: Overclassing Minority Classification

sum_1 = sum(y_train==1)
sum_0 = sum(y_train==0)

print('Total Data', len(y_train))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

print("\nPerforming SMOTE on training data set\n")
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

sum_1 = sum(y_train_res==1)
sum_0 = sum(y_train_res==0)

print('Total Data', len(y_train_res))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

Total Data 246008
Imbalance: [-] 226132 [+] 19876

Performing SMOTE on training data set

Total Data 452264
Imbalance: [-] 226132 [+] 226132


In [18]:
#amount of rows in input data
X_train_res.shape[0]

452264

In [19]:
#amount of columns in input data
len(X_train_res.columns)

245

In [20]:
model = DecisionTreeClassifier()
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [21]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8490154951790969


In [22]:
print(confusion_matrix(y_test,y_pred))

[[51393  5161]
 [ 4125   824]]


In [23]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  1.0
Decision Tree TEST score with  0.8490154951790969


In [24]:
auc = roc_auc_score(y_test, y_pred)
auc

0.5376201848449975

In [25]:
num_dataset.to_csv("full.csv")

In [ ]:
#cross_validate
cv_results = cross_validate(model, X_train_res, y_train_res, cv=10)

In [ ]:
#score with cross validation
np.mean(cv_results['test_score'])

In [ ]:
param_grid = {"criterion": ['entropy', 'gini'],
"min_samples_split": [5,10,15],
"max_depth": [2,3,5],
"min_samples_leaf": [5,10,15],
"max_leaf_nodes": [5,10,15],
}
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train_res, y_train_res)
CV_rfc.best_params_

In [ ]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=2, 
                               max_leaf_nodes=5, min_samples_leaf=5, min_samples_split=5)
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
auc = roc_auc_score(y_test, y_pred)
auc